### Summary

Code below iterates through our training data, converts each image into an array and resizes them. Final output should be an array of arrays, with each pixel represented as numbers.

In [25]:
import os
from PIL import Image
import numpy as np

STANDARD_SIZE = (300, 300)   #Desired resized size

def img_to_matrix(filename):   #Conver image to pixel representations
    """
    takes a filename and turns it into a numpy array of RGB pixels
    """
    img = Image.open(filename)
    img = img.resize(STANDARD_SIZE)
    img = list(img.getdata())
    img = map(list, img)
    img = np.array(img)
    return img


rootdir = (os.getcwd()) #Location of TrainImages folder. os.getcwd gets the current directory.
image_array = []                 #Change folder to appropriate directory.

for subdir, dirs, files in os.walk(rootdir):
    for file in files:
        if file.endswith("jpg"):
            file = img_to_matrix(os.path.join(subdir, file))
            image_array.append(file)
        elif file.endswith("png"):
            file = img_to_matrix(os.path.join(subdir, file))
            image_array.append(file)
            